# CorrelAid for Klima*Kollektiv
Aug 17, 2024

by Nicolas Fröhlich, partly based on Florian Detsch and ChatGPT -- translated to Python by ChatGPT

In [2]:
# !pip install geopandas
# !pip install folium
# !pip install requests

     |████████████████████████████████| 108 kB 1.5 MB/s eta 0:00:01


In [6]:
import pandas as pd
import geopandas as gpd
import folium
import requests
from shapely.geometry import Polygon

In [4]:
# ---- Scrape consumers and identify large ones ----

# Read geojson of Wasserverbraucher (only run once, please)
url = "https://www.klaerwerk-krefeld.org/wasserbuch/utm.php"
data = requests.get(url).json()

In [30]:
# Save to GeoDataFrame
consumer_all = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

# Save locally as GeoJSON
consumer_all.to_file("consumer_all.geojson", driver='GeoJSON')

# Convert 'size' to numeric
consumer_all['size'] = pd.to_numeric(consumer_all['size'], errors='coerce')

# Identify large consumers (more than 1 mio m3/a of water granted)
consumer = consumer_all[consumer_all['size'] > 1000000]


In [49]:
# ---- Spatial subset with bounding box ----

# Define coordinates of the rectangle [Köln, Mettmann, Roermond, Aachen]
coords = [(5.9512135386, 50.7489864309),
          (7.0077544451, 50.7489864309),
          (7.0077544451, 51.2670019108),
          (5.9512135386, 51.2670019108),
          (5.9512135386, 50.7489864309)]  # Close the polygon by repeating the first point

# Create a polygon from the coordinates
polygon = Polygon(coords)

# Create a GeoDataFrame for the polygon
polygon_gdf = gpd.GeoDataFrame(index=[0], crs=consumer.crs, geometry=[polygon])

# Find the points that intersect with the polygona
consumer_within = consumer[consumer.intersects(polygon)]

In [50]:
# ---- Visualization ----

# Create a base map
m = folium.Map(location=[51.0, 6.5], zoom_start=8)

# Add bounding box polygon with red border
folium.GeoJson(polygon_gdf,
               style_function=lambda x: {'color': 'red', 'weight': 2}
              ).add_to(m)

# Add consumer points to the map
for idx, row in consumer_within.iterrows():
    title_part = row['title'].split('/', 1)[1] if '/' in row['title'] else row['title']
    popup_text = f"{title_part} consumes {row['size']} litres of water"
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        popup=folium.Popup(html=popup_text, max_width=200),
        color='blue',
        fill=True
    ).add_to(m)

In [51]:
# save the map as hmtl
m.save('large_consumers.html')

In [52]:
m